# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.  Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. 

## Откроем и изучим файл

In [72]:
# импортируем библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score

In [73]:
# откроем файл и выведем первые строки на экран
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [74]:
# изучим общую информацию о файле
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Файл представляет собой таблицу из 5 столбцов и 3214 строк. Столбец is_ultra содержит в себе целевой признак. Будем решать задачу бинарной классификации.

In [75]:
# запишем обучающую выборку в переменную features
features = df.drop(['is_ultra'], axis=1)
# запишем целевой признак в переменную target
target = df['is_ultra']

## Разбобьём данные на выборки

Разделим исходные данные на обучающую и валидационную выборки.

In [76]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.75, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features, target, test_size=0.5, random_state=12345)

## Исследуем модели

1. Дерево решений.

In [77]:
decision_tree_best_model = None
decision_tree_best_result = 0
for depth in range(1, 6):
    decision_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # задаем глубину дерева
    decision_tree_model.fit(features_train,target_train) # обучаем модель
    decision_tree_predictions = decision_tree_model.predict(features_valid) # получим предсказания модели
    decision_tree_result = accuracy_score(target_valid, decision_tree_predictions) # посчитаем качество модели
    if decision_tree_result > decision_tree_best_result:
        decision_tree_best_model = decision_tree_model
        decision_tree_best_result = decision_tree_result
        max_depth = depth
print("max_depth =", max_depth)
print("Accuracy лучшей модели дерева решений:", decision_tree_best_result)

max_depth = 5
Accuracy лучшей модели дерева решений: 0.8164281269446173


2. Случайный лес.

In [78]:
random_forest_best_model = None
random_forest_best_result = 0
for est in range(1, 11):
    # задаем количество деревьев
    random_forest_model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    # обучаем модель на тренировочной выборке
    random_forest_model.fit(features_train,target_train)
    # считаем качество модели на валидационной выборке
    random_forest_result = random_forest_model.score(features_valid,target_valid)
    if random_forest_result > random_forest_best_result:
        random_forest_best_model = random_forest_model # сохраним наилучшую модель
        #  сохраним наилучшее значение метрики accuracy на валидационных данных
        random_forest_best_result = random_forest_result 
        estimators = est  
print("estimators =", estimators)
print("Accuracy наилучшей модели случайного леса на валидационной выборке:", random_forest_best_result)

estimators = 9
Accuracy наилучшей модели случайного леса на валидационной выборке: 0.8917237087741132


3. Логистическая регрессия. 

In [79]:
# задаем random_state, solver (алгоритм) и max_iter (максимальное количество итераций)
logistic_regression_model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
logistic_regression_model.fit(features_train, target_train) 
logistic_regression_model.predict(features_valid) 
logistic_regression_result = model.score(features_valid, target_valid) 
print("Accuracy модели логистической регрессии на валидационной выборке:", logistic_regression_result)

Accuracy модели логистической регрессии на валидационной выборке: 0.8917237087741132


Наилучшая модель на валидационной выборке random_forest_best_model с количеством деревьев 9 - показала accuracy 0.8917237087741132. Проверим её на тестовой выборке. 

## Проверим модель на тестовой выборке

Проверим модель  random_forest_best_model на тестовой выборке. 

In [81]:
# выбираем наилучшую модель с заданными параметрами
model = RandomForestClassifier(random_state=12345, n_estimators= 9) 
# обучаем модель
model.fit(features_train,target_train)
# проверяем accuracy на тестовой выборке
print("Accuracy наилучшей модели на тестовой выборке:", model.score(features_test,target_test))

Accuracy наилучшей модели на тестовой выборке: 0.761045426260112


Доля правильных ответов наилучшей модели на тестовой выборке показывает меньшее значение, чем на валидационной, но оно находится в пределах заданного порога - не менее 0.75. 